# 8-puzzle problem using misplaced tiles and manhattan distance


In [7]:
import heapq

class PuzzleState:
    def __init__(self, board, parent=None, move="", depth=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth  # g(n)
        self.zero_position = self.board.index(0)

    # Number of misplaced tiles heuristic (h1)
    def misplaced_tiles(self, goal):
        return sum([1 if self.board[i] != goal[i] and self.board[i] != 0 else 0 for i in range(9)])

    # Manhattan distance heuristic (h2)
    def manhattan_distance(self, goal):
        distance = 0
        for i in range(1, 9):  # excluding the empty tile (0)
            current_pos = self.board.index(i)
            goal_pos = goal.index(i)
            current_row, current_col = divmod(current_pos, 3)
            goal_row, goal_col = divmod(goal_pos, 3)
            distance += abs(current_row - goal_row) + abs(current_col - goal_col)
        return distance

    def generate_children(self):
        children = []
        x, y = divmod(self.zero_position, 3)  # Get the coordinates of the blank tile

        moves = [
            (-1, 0, "Up"),  # Move up
            (1, 0, "Down"),  # Move down
            (0, -1, "Left"),  # Move left
            (0, 1, "Right"),  # Move right
        ]

        for dx, dy, direction in moves:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_zero_pos = new_x * 3 + new_y
                new_board = self.board[:]
                # Swap the blank tile with the target tile
                new_board[self.zero_position], new_board[new_zero_pos] = new_board[new_zero_pos], new_board[self.zero_position]
                children.append(PuzzleState(new_board, parent=self, move=direction, depth=self.depth + 1))

        return children

    def __lt__(self, other):
        return self.depth < other.depth

    def print_board(self):
        """Helper function to print the board in a 3x3 format."""
        for i in range(3):
            print(" ".join(str(x) for x in self.board[i*3:(i+1)*3]))
        print()  # Print a newline for better readability

def a_star(start, goal, heuristic='manhattan'):
    open_set = []
    heapq.heappush(open_set, (0, start))
    closed_set = set()

    while open_set:
        f_n, current_state = heapq.heappop(open_set)
        closed_set.add(tuple(current_state.board))

        # Calculate heuristic value (h(n)) for current state
        if heuristic == 'misplaced':
            h_n = current_state.misplaced_tiles(goal)
        elif heuristic == 'manhattan':
            h_n = current_state.manhattan_distance(goal)
        else:
            raise ValueError("Invalid heuristic! Choose 'misplaced' or 'manhattan'.")

        # Check if the goal state has been reached
        if current_state.board == goal:
            # Reconstruct the path and print h(n), g(n), f(n) for the final path
            path = []
            solution_states = []
            while current_state.parent:
                path.append(current_state.move)
                solution_states.append(current_state)  # Track the states leading to the solution
                current_state = current_state.parent
            solution_states.append(current_state)  # Include the start state

            # Reverse the states to print in the correct order
            solution_states.reverse()
            path.reverse()

            print("\nSolution path:")
            for state in solution_states:
                g_n = state.depth
                h_n = state.misplaced_tiles(goal) if heuristic == 'misplaced' else state.manhattan_distance(goal)
                f_n = g_n + h_n
                move = state.move if state.move else "Start"
                print(f"Move: {move}, g(n): {g_n}, h(n): {h_n}, f(n): {f_n}")
                state.print_board()

            # Print the total number of steps
            print(f"Total number of steps to solve: {len(path)}")

            return path  # Return the path of moves

        # Generate children and add them to the open set
        for child in current_state.generate_children():
            if tuple(child.board) in closed_set:
                continue

            # Recalculate heuristic value for child
            h_n = child.misplaced_tiles(goal) if heuristic == 'misplaced' else child.manhattan_distance(goal)
            f_n = child.depth + h_n  # f(n) = g(n) + h(n)
            heapq.heappush(open_set, (f_n, child))

    return None  # No solution found

def input_board(prompt):
    board = []
    print(prompt)
    for i in range(3):
        row = input(f"Enter row {i+1} (3 numbers separated by space): ").split()
        board.extend([int(x) for x in row])
    return board

# Main program
if __name__ == "__main__":
    print("A* 8-Puzzle Solver")

    # Take user input for the start state
    start_board = input_board("Enter the start state (use 0 for the blank):")

    # Define the fixed goal state
    goal_board = [0, 1, 2, 3, 4, 5, 6, 7, 8]

    start_state = PuzzleState(start_board)

    print("\nSolving with misplaced tiles heuristic:")
    path_misplaced = a_star(start_state, goal_board, heuristic='misplaced')
    if not path_misplaced:
        print("No solution found.")

    print("\nSolving with Manhattan distance heuristic:")
    path_manhattan = a_star(start_state, goal_board, heuristic='manhattan')
    if not path_manhattan:
        print("No solution found.")


A* 8-Puzzle Solver
Enter the start state (use 0 for the blank):
Enter row 1 (3 numbers separated by space): 5 4 0
Enter row 2 (3 numbers separated by space): 6 1 8
Enter row 3 (3 numbers separated by space): 7 3 2

Solving with misplaced tiles heuristic:

Solution path:
Move: Start, g(n): 0, h(n): 8, f(n): 8
5 4 0
6 1 8
7 3 2

Move: Left, g(n): 1, h(n): 8, f(n): 9
5 0 4
6 1 8
7 3 2

Move: Down, g(n): 2, h(n): 7, f(n): 9
5 1 4
6 0 8
7 3 2

Move: Right, g(n): 3, h(n): 7, f(n): 10
5 1 4
6 8 0
7 3 2

Move: Down, g(n): 4, h(n): 7, f(n): 11
5 1 4
6 8 2
7 3 0

Move: Left, g(n): 5, h(n): 7, f(n): 12
5 1 4
6 8 2
7 0 3

Move: Left, g(n): 6, h(n): 6, f(n): 12
5 1 4
6 8 2
0 7 3

Move: Up, g(n): 7, h(n): 5, f(n): 12
5 1 4
0 8 2
6 7 3

Move: Up, g(n): 8, h(n): 5, f(n): 13
0 1 4
5 8 2
6 7 3

Move: Right, g(n): 9, h(n): 6, f(n): 15
1 0 4
5 8 2
6 7 3

Move: Right, g(n): 10, h(n): 6, f(n): 16
1 4 0
5 8 2
6 7 3

Move: Down, g(n): 11, h(n): 5, f(n): 16
1 4 2
5 8 0
6 7 3

Move: Left, g(n): 12, h(n): 5, f(n

In [8]:
import heapq

class PuzzleState:
    def __init__(self, board, parent=None, move="", depth=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth  # g(n) = the cost so far (depth)
        self.zero_position = self.board.index(0)

    # Misplaced tiles heuristic (h1)
    def misplaced_tiles(self, goal):
        return sum(1 for i in range(9) if self.board[i] != goal[i] and self.board[i] != 0)

    # Manhattan distance heuristic (h2)
    def manhattan_distance(self, goal):
        distance = 0
        for i in range(1, 9):  # Skip the empty tile (0)
            current_pos = self.board.index(i)
            goal_pos = goal.index(i)
            current_row, current_col = divmod(current_pos, 3)
            goal_row, goal_col = divmod(goal_pos, 3)
            distance += abs(current_row - goal_row) + abs(current_col - goal_col)
        return distance

    # Generate child states from valid moves
    def generate_children(self):
        children = []
        x, y = divmod(self.zero_position, 3)

        moves = [
            (-1, 0, "Up"),   # Move up
            (1, 0, "Down"),  # Move down
            (0, -1, "Left"), # Move left
            (0, 1, "Right")  # Move right
        ]

        for dx, dy, direction in moves:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_zero_pos = new_x * 3 + new_y
                new_board = self.board[:]
                new_board[self.zero_position], new_board[new_zero_pos] = new_board[new_zero_pos], new_board[self.zero_position]
                children.append(PuzzleState(new_board, parent=self, move=direction, depth=self.depth + 1))

        return children

    def __lt__(self, other):
        return self.depth < other.depth

def a_star(start, goal, heuristic='manhattan'):
    open_set = []
    heapq.heappush(open_set, (0, start))
    closed_set = set()

    while open_set:
        f_n, current_state = heapq.heappop(open_set)
        closed_set.add(tuple(current_state.board))

        # Calculate heuristic value (h(n)) for current state
        if heuristic == 'misplaced':
            h_n = current_state.misplaced_tiles(goal)
        elif heuristic == 'manhattan':
            h_n = current_state.manhattan_distance(goal)
        else:
            raise ValueError("Invalid heuristic! Choose 'misplaced' or 'manhattan'.")

        # Check if the goal state has been reached
        if current_state.board == goal:
            # Reconstruct the path and print h(n), g(n), f(n) for the final path
            path = []
            solution_states = []
            while current_state.parent:
                path.append(current_state.move)
                solution_states.append(current_state)  # Track the states leading to the solution
                current_state = current_state.parent
            solution_states.append(current_state)  # Include the start state

            # Reverse the states to print in the correct order
            solution_states.reverse()
            path.reverse()

            print("\nSolution path:")
            for state in solution_states:
                g_n = state.depth
                h_n = state.misplaced_tiles(goal) if heuristic == 'misplaced' else state.manhattan_distance(goal)
                f_n = g_n + h_n
                move = state.move if state.move else "Start"
                print(f"Move: {move}, g(n): {g_n}, h(n): {h_n}, f(n): {f_n}")

            # Print the total number of steps
            print(f"Total number of steps to solve: {len(path)}")

            return path  # Return the path of moves

        # Generate children and add them to the open set
        for child in current_state.generate_children():
            if tuple(child.board) in closed_set:
                continue

            # Recalculate heuristic value for child
            h_n = child.misplaced_tiles(goal) if heuristic == 'misplaced' else child.manhattan_distance(goal)
            f_n = child.depth + h_n  # f(n) = g(n) + h(n)
            heapq.heappush(open_set, (f_n, child))

    return None  # No solution found

def input_board(prompt):
    board = []
    print(prompt)
    for i in range(3):
        row = input(f"Enter row {i+1} (3 numbers separated by space): ").split()
        board.extend([int(x) for x in row])
    return board

# Main program
if __name__ == "__main__":
    print("A* 8-Puzzle Solver")

    # Take user input for the start state
    start_board = input_board("Enter the start state (use 0 for the blank):")

    # Define the fixed goal state
    goal_board = [0, 1, 2, 3, 4, 5, 6, 7, 8]

    start_state = PuzzleState(start_board)

    print("\nSolving with misplaced tiles heuristic:")
    path_misplaced = a_star(start_state, goal_board, heuristic='misplaced')
    if path_misplaced:
        print("Moves:", path_misplaced)
        print(f"Total number of steps: {len(path_misplaced)}")
    else:
        print("No solution found.")

    print("\nSolving with Manhattan distance heuristic:")
    path_manhattan = a_star(start_state, goal_board, heuristic='manhattan')
    if path_manhattan:
        print("Moves:", path_manhattan)
        print(f"Total number of steps: {len(path_manhattan)}")
    else:
        print("No solution found.")


A* 8-Puzzle Solver
Enter the start state (use 0 for the blank):
Enter row 1 (3 numbers separated by space): 5 4 0
Enter row 2 (3 numbers separated by space): 6 1 8
Enter row 3 (3 numbers separated by space): 7 3 2

Solving with misplaced tiles heuristic:

Solution path:
Move: Start, g(n): 0, h(n): 8, f(n): 8
Move: Left, g(n): 1, h(n): 8, f(n): 9
Move: Down, g(n): 2, h(n): 7, f(n): 9
Move: Right, g(n): 3, h(n): 7, f(n): 10
Move: Down, g(n): 4, h(n): 7, f(n): 11
Move: Left, g(n): 5, h(n): 7, f(n): 12
Move: Left, g(n): 6, h(n): 6, f(n): 12
Move: Up, g(n): 7, h(n): 5, f(n): 12
Move: Up, g(n): 8, h(n): 5, f(n): 13
Move: Right, g(n): 9, h(n): 6, f(n): 15
Move: Right, g(n): 10, h(n): 6, f(n): 16
Move: Down, g(n): 11, h(n): 5, f(n): 16
Move: Left, g(n): 12, h(n): 5, f(n): 17
Move: Left, g(n): 13, h(n): 5, f(n): 18
Move: Down, g(n): 14, h(n): 6, f(n): 20
Move: Right, g(n): 15, h(n): 7, f(n): 22
Move: Right, g(n): 16, h(n): 7, f(n): 23
Move: Up, g(n): 17, h(n): 6, f(n): 23
Move: Left, g(n): 18, 